In [1]:
from glob import glob
import pandas as pd
import numpy as np
import os
import json
from collections import defaultdict

In [3]:
# df_movies = pd.read_csv('dataset/tmdb_5000_movies.csv')
# df_movies = df_movies.dropna(subset=['overview', 'release_date', 'runtime'])
# df_movies.isnull().sum()
# df_movies.to_csv("dataset/tmdb_5000_movies_nonull.csv")

In [4]:
df = pd.read_csv('dataset/tmdb_5000_movies_nonull.csv')
# df_movies.loc[:,['id', 'keywords', 'genres', 'title']]
# df_credits = pd.read_csv('dataset/tmdb_5000_credits.csv')

df = df[['budget', 'genres', 'keywords', 'original_language'
                      , 'overview', 'popularity', 'production_companies'
                     , 'production_countries', 'release_date', 'revenue'
                     , 'runtime', 'spoken_languages', 'tagline', 'title'
                     , 'vote_average', 'vote_count']]
display(df.sort_values(by='release_date', ascending=False).head())
# display(df_credits.head())

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count
4253,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...",[],en,"In 1979, an Indian family moves to America wit...",0.710870,[],[],2017-02-03,0,102.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",It’s better to stand out than to fit in.,Growing Up Smith,7.4,7
3407,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...","[{""id"": 1415, ""name"": ""small town""}, {""id"": 46...",en,Set in a small town near the North Pole where ...,3.139294,[],[],2016-10-02,0,96.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",NaN,Two Lovers and a Bear,6.8,27
3301,8000000,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 1650, ""name"": ""cook""}, {""id"": 6054, ""n...",en,A unique friendship develops when a little gir...,7.828459,"[{""name"": ""Envision Media Arts"", ""id"": 19456},...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-09-16,0,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",He was the one person she could always count on.,Mr. Church,7.0,129
3248,0,"[{""id"": 12, ""name"": ""Adventure""}]","[{""id"": 3405, ""name"": ""blow job""}, {""id"": 1173...",en,When his hard-earned kicks get snatched by a l...,3.467923,"[{""name"": ""Bystorm Films"", ""id"": 2903}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-09-09,0,80.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",They aren't just shoes,Kicks,7.5,18
4716,8500000,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 2831, ""name"": ""slavery""}]",en,"Nat Turner, a former slave in America, leads a...",9.452808,"[{""name"": ""Phantom Four"", ""id"": 423}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-09-09,15861566,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Untold Story of Nat Turner,The Birth of a Nation,6.5,178


In [5]:
def convert_list(cell):
    """convert the json format to a list of categories"""
    kw_list = []
    for kw in json.loads(cell):
        kw_list.append(kw['name'])
    return kw_list


def larger_n(col, n):
    """filter the column"""
    keywords = defaultdict(int)
    for row in df_movies[col]:
        row = json.loads(row)
        for entry in row:
            keywords[entry['name']] += 1
    kw_cnt = sorted(keywords.items(), key=lambda x: -x[1])
    return [kw[0] for kw in kw_cnt if kw[1] >= n]

df_movies = df

larger_kw = larger_n('keywords', 30)
larger_genre = larger_n('genres', 0)
larger_pc = larger_n('production_companies', 5)
larger_pcount = larger_n('production_countries', 3)
larger_lang = larger_n('spoken_languages', 10)

df_movies['keywords'] = df_movies['keywords'].apply(
    convert_list).apply(lambda cell: [kw for kw in cell if kw in larger_kw])
# df_movies['keywords'] = df_movies['keywords'].apply(convert_list)
df_movies['genres'] = df_movies['genres'].apply(convert_list).apply(
    lambda cell: [kw for kw in cell if kw in larger_genre])
df_movies['production_companies'] = df_movies['production_companies'].apply(convert_list).apply(
    lambda cell: [kw for kw in cell if kw in larger_pc])
df_movies['production_countries'] = df_movies['production_countries'].apply(convert_list).apply(
    lambda cell: [kw for kw in cell if kw in larger_pcount])
df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(convert_list).apply(
    lambda cell: [kw for kw in cell if kw in larger_lang])

df_movies

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]","[future, alien, soldier, battle, 3d]",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[English, Español]",Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]","[love of one's life, ship, aftercreditsstinger]",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films]",[United States of America],2007-05-19,961000000,169.0,[English],"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[Action, Adventure, Crime]","[spy, based on novel, sequel]",en,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,"[Français, English, Español, Italiano, Deutsch]",A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[Action, Crime, Drama, Thriller]","[terrorist, secret identity, superhero]",en,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,[English],The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[Action, Adventure, Science Fiction]","[based on novel, alien, escape, 3d]",en,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,[English],"Lost in our world, found in another.",John Carter,6.1,2124
5,258000000,"[Fantasy, Action, Adventure]","[love of one's life, marvel comic, sequel, sup...",en,The seemingly invincible Spider-Man goes up ag...,115.699814,"[Columbia Pictures, Laura Ziskin Productions, ...",[United States of America],2007-05-01,890871626,139.0,"[English, Français]",The battle within.,Spider-Man 3,5.9,3576
6,260000000,"[Animation, Family]","[hostage, magic, musical, duringcreditsstinger]",en,When the kingdom's most wanted-and most charmi...,48.681969,"[Walt Disney Pictures, Walt Disney Animation S...",[United States of America],2010-11-24,591794936,100.0,[English],They're taking adventure to new lengths.,Tangled,7.4,3330
7,280000000,"[Action, Adventure, Science Fiction]","[marvel comic, sequel, superhero, based on com...",en,When Tony Stark tries to jumpstart a dormant p...,134.279229,"[Marvel Studios, Revolution Sun Studios]",[United States of America],2015-04-22,1405403694,141.0,[English],A New Age Has Come.,Avengers: Age of Ultron,7.3,6767
8,250000000,"[Adventure, Fantasy, Family]","[witch, magic]",en,"As Harry begins his sixth year at Hogwarts, he...",98.885637,"[Warner Bros., Heyday Films]","[United Kingdom, United States of America]",2009-07-07,933959197,153.0,[English],Dark Secrets Revealed,Harry Potter and the Half-Blood Prince,7.4,5293
9,250000000,"[Action, Adventure, Fantasy]","[superhero, based on comic book, revenge]",en,Fearing the actions of a god-like Super Hero l...,155.790452,"[DC Comics, Atlas Entertainment, Warner Bros.,...",[United States of America],2016-03-23,873260194,151.0,[English],Justice or revenge,Batman v Superman: Dawn of Justice,5.7,7004


# Feature Engineering

## Overview -- Tfidf

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

df_movies = df_movies.loc[df_movies.overview.apply(lambda x: len(x)>10)]
df_movies = df_movies.reset_index(drop=True)

v = TfidfVectorizer(stop_words='english',binary=True, ngram_range=(1, 2),min_df=0.001)
x = v.fit_transform(df_movies['overview'])

overview = pd.DataFrame(x.toarray(), columns=list(range(x.toarray().shape[1])))

## budget + revenue

In [7]:
df_movies.loc[(df_movies.budget<100)].shape

(1057, 16)

for these, use revenue + overview + popularity + release_date to predict the budget

In [8]:
df_movies.loc[(df_movies.budget<100) & (df_movies.revenue>100)].shape

(145, 16)

In [9]:
def predict_budget1(dataset):
    from sklearn import ensemble
    
    data_p = dataset[['revenue','popularity','budget']]
    v = TfidfVectorizer(stop_words='english',binary=True, ngram_range=(1, 2),min_df=0.001)
    x = v.fit_transform(dataset['overview'])
    overview = pd.DataFrame(x.toarray(), columns=list(range(x.toarray().shape[1])))
    data_p = pd.concat([data_p, overview], axis=1)
    
    x_train = data_p.loc[~(data_p.budget<100) & (data_p.revenue>100)].drop('budget', 1)
    y_train = data_p.loc[~(data_p.budget<100) & (data_p.revenue>100)]['budget']
    x_test = data_p.loc[(data_p.budget<100) & (data_p.revenue>100)].drop('budget', 1)
    
    rfr = ensemble.RandomForestRegressor(random_state=42)
    rfr.fit(x_train,y_train)
    
    y_test = rfr.predict(x_test)
    return y_test

df_movies.loc[(df_movies.budget<100) & (df_movies.revenue>100), 'budget'] = predict_budget1(df_movies)

/home/julia/MSDS/Fall18/msds621/miniconda/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [10]:
df_movies.loc[(df_movies.budget<100) & (df_movies.revenue>100)].shape

(0, 16)

for others, use overview+popularity+release_date to predict budget

In [11]:
df_movies.loc[(df_movies.budget<100)].shape

(912, 16)

In [12]:
def predict_budget2(dataset):
    from sklearn import ensemble
    data_p = dataset[['popularity','budget']]
    v = TfidfVectorizer(stop_words='english',binary=True, ngram_range=(1, 2),min_df=0.001)
    x = v.fit_transform(dataset['overview'])
    overview = pd.DataFrame(x.toarray(), columns=list(range(x.toarray().shape[1])))
    data_p = pd.concat([data_p, overview], axis=1)
    
    x_train = data_p.loc[~(data_p.budget<100)].drop('budget', 1)
    y_train = data_p.loc[~(data_p.budget<100)]['budget']
    x_test = data_p.loc[(data_p.budget<100) & (df_movies.revenue<=100)].drop('budget', 1)
    
    rfr = ensemble.RandomForestRegressor(random_state=42)
    rfr.fit(x_train,y_train)
    print(x_test.shape)
    y_test = rfr.predict(x_test)
    return y_test

df_movies.loc[(df_movies.budget<100) & (df_movies.revenue<=100), 'budget'] = predict_budget2(df_movies)

/home/julia/MSDS/Fall18/msds621/miniconda/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


(912, 5561)


In [13]:
df_movies.loc[(df_movies.budget<100)].shape

(0, 16)

## runtime

## keywords + tagline

## original_language + spoken_language
1) original language is the language of production. 2) spoken language is the language of production too and add the languages in case of multi-language in the movie.

## production_companies

## production_countries

## release_date

## popularity + vote_average + vote_count

In [14]:
df_movies.loc[df_movies['popularity']==0]

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count


# Result-dataset

In [15]:
df_movies = df_movies.reset_index(drop=True)

In [16]:
df_movies

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count
0,2.370000e+08,"[Action, Adventure, Fantasy, Science Fiction]","[future, alien, soldier, battle, 3d]",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[English, Español]",Enter the World of Pandora.,Avatar,7.2,11800
1,3.000000e+08,"[Adventure, Fantasy, Action]","[love of one's life, ship, aftercreditsstinger]",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films]",[United States of America],2007-05-19,961000000,169.0,[English],"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,2.450000e+08,"[Action, Adventure, Crime]","[spy, based on novel, sequel]",en,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,"[Français, English, Español, Italiano, Deutsch]",A Plan No One Escapes,Spectre,6.3,4466
3,2.500000e+08,"[Action, Crime, Drama, Thriller]","[terrorist, secret identity, superhero]",en,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,[English],The Legend Ends,The Dark Knight Rises,7.6,9106
4,2.600000e+08,"[Action, Adventure, Science Fiction]","[based on novel, alien, escape, 3d]",en,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,[English],"Lost in our world, found in another.",John Carter,6.1,2124
5,2.580000e+08,"[Fantasy, Action, Adventure]","[love of one's life, marvel comic, sequel, sup...",en,The seemingly invincible Spider-Man goes up ag...,115.699814,"[Columbia Pictures, Laura Ziskin Productions, ...",[United States of America],2007-05-01,890871626,139.0,"[English, Français]",The battle within.,Spider-Man 3,5.9,3576
6,2.600000e+08,"[Animation, Family]","[hostage, magic, musical, duringcreditsstinger]",en,When the kingdom's most wanted-and most charmi...,48.681969,"[Walt Disney Pictures, Walt Disney Animation S...",[United States of America],2010-11-24,591794936,100.0,[English],They're taking adventure to new lengths.,Tangled,7.4,3330
7,2.800000e+08,"[Action, Adventure, Science Fiction]","[marvel comic, sequel, superhero, based on com...",en,When Tony Stark tries to jumpstart a dormant p...,134.279229,"[Marvel Studios, Revolution Sun Studios]",[United States of America],2015-04-22,1405403694,141.0,[English],A New Age Has Come.,Avengers: Age of Ultron,7.3,6767
8,2.500000e+08,"[Adventure, Fantasy, Family]","[witch, magic]",en,"As Harry begins his sixth year at Hogwarts, he...",98.885637,"[Warner Bros., Heyday Films]","[United Kingdom, United States of America]",2009-07-07,933959197,153.0,[English],Dark Secrets Revealed,Harry Potter and the Half-Blood Prince,7.4,5293
9,2.500000e+08,"[Action, Adventure, Fantasy]","[superhero, based on comic book, revenge]",en,Fearing the actions of a god-like Super Hero l...,155.790452,"[DC Comics, Atlas Entertainment, Warner Bros.,...",[United States of America],2016-03-23,873260194,151.0,[English],Justice or revenge,Batman v Superman: Dawn of Justice,5.7,7004


In [17]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

genres_dummies = pd.DataFrame(mlb.fit_transform(
    df_movies['genres']), columns=mlb.classes_).add_prefix('genre_')
keywords_dummies = pd.DataFrame(mlb.fit_transform(
    df_movies['keywords']), columns=mlb.classes_).add_prefix('kw_')
df_new = pd.concat([
    df_movies.loc[:, ['budget']],
    genres_dummies, keywords_dummies,
    overview], axis=1)
index_genre = df_new.columns.to_series().apply(lambda x: str(x).startswith('genre_'))
df_y = df_new.loc[:, index_genre]
df_X = df_new.drop(
    df_new.columns[index_genre], inplace=False, axis=1)

In [18]:
df_X

,budget,kw_1970s,kw_3d,kw_adultery,kw_aftercreditsstinger,kw_airplane,kw_alcohol,kw_alien,kw_army,kw_assassin,...,5550,5551,5552,5553,5554,5555,5556,5557,5558,5559
0,2.370000e+08,0,1,0,0,0,0,1,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,3.000000e+08,0,0,0,1,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,2.450000e+08,0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,2.500000e+08,0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,2.600000e+08,0,1,0,0,0,0,1,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5,2.580000e+08,0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6,2.600000e+08,0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7,2.800000e+08,0,1,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
8,2.500000e+08,0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9,2.500000e+08,0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [19]:
df_y

,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,genre_Family,genre_Fantasy,genre_Foreign,genre_History,genre_Horror,genre_Music,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western
0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
9,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


## get dummies

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
genres_dummies = pd.DataFrame(mlb.fit_transform(
    df_movies['genres']), columns=mlb.classes_).add_prefix('genre_')
keywords_dummies = pd.DataFrame(mlb.fit_transform(
    df_movies['keywords']), columns=mlb.classes_).add_prefix('kw_')
df_new = pd.concat([
    #df_movies.loc[:, ['budget']],
    genres_dummies, keywords_dummies,
    train_features], axis=1)

df_y = df_new.loc[:, df_new.columns.str.startswith('genre_')]
df_X = df_new.drop(
    df_new.columns[df_new.columns.str.startswith('genre_')], inplace=False, axis=1)

NameError: name 'train_features' is not defined

In [ ]:
df_X

In [21]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(df_X, df_y, test_size=0.33, random_state=42)

In [58]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier

classifier = BinaryRelevance(RandomForestClassifier())
%time classifier.fit(train_X, train_y)
predictions = classifier.predict(test_X)

from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

/home/julia/MSDS/Fall18/msds621/miniconda/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/julia/MSDS/Fall18/msds621/miniconda/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/julia/MSDS/Fall18/msds621/miniconda/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/julia/MSDS/Fall18/msds621/miniconda/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 

CPU times: user 53.4 s, sys: 1.94 s, total: 55.3 s
Wall time: 55.4 s


0.09027777777777778

In [27]:
predictions_dense = predictions.todense()

for i in range(predictions.shape[1]):
    print(accuracy_score(np.asarray(test_y)[:, i], np.asarray(predictions_dense)[:, i]))

0.803030303030303
0.8453282828282829
0.9602272727272727
0.7077020202020202
0.8535353535353535
0.9779040404040404
0.6616161616161617
0.9008838383838383
0.9147727272727273
0.9955808080808081
0.9665404040404041
0.913510101010101
0.9532828282828283
0.9185606060606061
0.8282828282828283
0.9210858585858586
0.9974747474747475
0.7481060606060606
0.9671717171717171
0.9835858585858586


In [57]:
# RF can handle multilabel directly


rf = RandomForestClassifier()
%time rf.fit(train_X, train_y)
predictions = rf.predict(test_X)

from sklearn.metrics import accuracy_score
print(f"Total accuracy: {accuracy_score(test_y, predictions)}")

#predictions_dense = predictions#.todense()

for i in range(predictions.shape[1]):
    print(accuracy_score(test_y.values[:, i], predictions[:, i]))

/home/julia/MSDS/Fall18/msds621/miniconda/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 1.95 s, sys: 80 ms, total: 2.03 s
Wall time: 2.04 s
Total accuracy: 0.09154040404040405
0.7916666666666666
0.8446969696969697
0.9595959595959596
0.67739898989899
0.8503787878787878
0.9804292929292929
0.6647727272727273
0.8996212121212122
0.9103535353535354
0.9955808080808081
0.9659090909090909
0.9002525252525253
0.9501262626262627
0.9210858585858586
0.8232323232323232
0.9046717171717171
0.9974747474747475
0.7582070707070707
0.9665404040404041
0.9835858585858586


In [56]:
# One vs Rest


from sklearn.multiclass import OneVsRestClassifier
rf = OneVsRestClassifier(RandomForestClassifier())
%time rf.fit(train_X, train_y)
predictions = rf.predict(test_X)

from sklearn.metrics import accuracy_score
print(accuracy_score(test_y, predictions))

for i in range(predictions.shape[1]):
    print(accuracy_score(test_y.values[:, i], predictions[:, i]))

/home/julia/MSDS/Fall18/msds621/miniconda/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/julia/MSDS/Fall18/msds621/miniconda/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/julia/MSDS/Fall18/msds621/miniconda/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/julia/MSDS/Fall18/msds621/miniconda/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 

CPU times: user 14.7 s, sys: 1.58 s, total: 16.3 s
Wall time: 16.3 s
0.09343434343434344
0.8093434343434344
0.8459595959595959
0.9583333333333334
0.7108585858585859
0.8503787878787878
0.9772727272727273
0.6609848484848485
0.8996212121212122
0.9147727272727273
0.9955808080808081
0.9659090909090909
0.9097222222222222
0.9507575757575758
0.9185606060606061
0.8238636363636364
0.9292929292929293
0.9974747474747475
0.7657828282828283
0.9671717171717171
0.9835858585858586
